In [1]:
from google.cloud import bigquery
import pandas as pd
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

client = bigquery.Client()

project_id_reviews = 'ingka-tugc-infra-prod'
dataset_id_reviews = 'eu_ai_content'
table_id_reviews = 'reviews'

table_ref_reviews = f'{project_id_reviews}.{dataset_id_reviews}.{table_id_reviews}'

project_id_prod_info = 'ingka-explainableai-tpu-dev'
dataset_id_prod_info = 'text_to_sql'
table_id_prod_info = 'item_dim'

table_ref_prod_info = f'{project_id_prod_info}.{dataset_id_prod_info}.{table_id_prod_info}'


query_all = f"""
    SELECT r.art_id, r.title, r.text, concat(r.title, '. ', r.text) as review_text, r.franchise, r.content_lang_code, r.updated_on, p.pa_name as product_area, type.name as type_name, p.item_name
    FROM {table_ref_prod_info} p 
    INNER JOIN {table_ref_reviews} r
    ON p.item_no = r.art_id
    WHERE p.item_no is not null AND franchise='set-11' AND content_lang_code = 'en'
"""

query_job = client.query(query_all)
df = query_job.to_dataframe()


In [2]:
target_dataset_size = 9000
min_reviews_threshold = 25
max_allowed_difference = 150

# 1. Compute total reviews in the dataset.
total_reviews = len(df)
print(f"Total number of reviews in dataset: {total_reviews}\n")



Total number of reviews in dataset: 9701897



In [3]:
# Load the CSV file containing article IDs to exclude.
exclude_articles_df = pd.read_csv('csv/GroundTruthProdArea10k.csv')  # Replace with your CSV file name.
exclude_article_ids = exclude_articles_df['article_id'].astype(str).tolist()
print(f"Number of article IDs to exclude: {len(exclude_article_ids)}")
# Filter the dataframe to exclude reviews belonging to the specified articles.
df = df[~df['art_id'].isin(exclude_article_ids)]

print(f"Filtered dataframe now contains {len(df)} reviews.")

Number of article IDs to exclude: 11202
Filtered dataframe now contains 9692679 reviews.


In [4]:
# 2. Compute total review count per product area.
product_area_counts = df.groupby('product_area').size().reset_index(name='review_count')

In [5]:
# 3. Compute the percentage share and the target number of reviews for each product area.
product_area_counts['percentage'] = product_area_counts['review_count'] / total_reviews
product_area_counts['target_reviews'] = product_area_counts['percentage'] * target_dataset_size

In [6]:
# 4. Group reviews by product_area and art_id to count reviews per article.
articles_review_counts = df.groupby(['product_area', 'art_id']).size().reset_index(name='article_review_count')

selected_articles = {}

In [7]:
# 5. For each product area, select an article based on the desired logic.
for _, row in product_area_counts.iterrows():
    product_area = row['product_area']
    allocated = row['target_reviews']
    
    # Filter articles in this product area.
    candidates = articles_review_counts[articles_review_counts['product_area'] == product_area].copy()
    
    if candidates.empty:
        continue  # Skip if no articles are found for this product area.
    
    # Case A: For product areas with allocated reviews below the minimum threshold.
    if allocated < min_reviews_threshold:
        # Look for any article with at least min_reviews_threshold reviews.
        valid_candidates = candidates[candidates['article_review_count'] >= min_reviews_threshold]
        if valid_candidates.empty:
            print(f"Ignoring product area '{product_area}' because no article has at least {min_reviews_threshold} reviews.")
            continue
        # Select the candidate with the maximum number of reviews.
        selected = valid_candidates.sort_values('article_review_count', ascending=False).iloc[0]
    else:
        # Case B: For product areas with allocated reviews >= minimum threshold,
        # choose the article whose review count is closest to the allocated count.
        candidates['diff'] = (candidates['article_review_count'] - allocated).abs()
        selected = candidates.sort_values('diff').iloc[0]
    
    # Check if the selected article's review count deviates by more than ±100 from the allocated reviews.
    if abs(selected['article_review_count'] - allocated) > max_allowed_difference:
        print(f"Skipping product area '{product_area}' because difference between allocated ({allocated:.2f}) and actual review count ({selected['article_review_count']}) is greater than {max_allowed_difference}.")
        continue
    
    selected_articles[product_area] = {
        'art_id': selected['art_id'],
        'actual_review_count': selected['article_review_count'],
        'allocated_reviews': allocated,
        'percentage': row['percentage']
    }

Skipping product area 'Airpurifiers & accessories' because difference between allocated (8.09) and actual review count (187) is greater than 150.
Skipping product area 'Artificial plants' because difference between allocated (8.68) and actual review count (397) is greater than 150.
Skipping product area 'Baby sleep & care furniture' because difference between allocated (5.80) and actual review count (195) is greater than 150.
Skipping product area 'Baby toys' because difference between allocated (3.46) and actual review count (307) is greater than 150.
Skipping product area 'Bakeware' because difference between allocated (10.82) and actual review count (749) is greater than 150.
Skipping product area 'Bath textile accessories' because difference between allocated (14.97) and actual review count (681) is greater than 150.
Skipping product area 'Bathroom around the sink' because difference between allocated (16.31) and actual review count (1161) is greater than 150.
Skipping product area

In [8]:
total_selected_reviews = sum(info['actual_review_count'] for info in selected_articles.values())
total_selected_articles = len(selected_articles)
print(f"Total number of reviews in the fine-tuning dataset: {total_selected_reviews}")
print(f"Total number of articles in the fine-tuning dataset: {total_selected_articles}\n")

Total number of reviews in the fine-tuning dataset: 10691
Total number of articles in the fine-tuning dataset: 103



In [9]:
# 6. Print the selected articles for each product area.
print("\nSelected Articles:")
for pa, info in selected_articles.items():
    print(f"Product Area: {pa}")
    print(f"  Selected Article ID: {info['art_id']}")
    print(f"  Actual review count: {info['actual_review_count']}")
    print(f"  Allocated reviews: {info['allocated_reviews']:.2f}")
    print(f"  Percentage of total reviews: {info['percentage']:.2%}\n")


Selected Articles:
Product Area: Armchairs
  Selected Article ID: 29535737
  Actual review count: 358
  Allocated reviews: 303.07
  Percentage of total reviews: 3.37%

Product Area: Artificial flowers
  Selected Article ID: 00335772
  Actual review count: 87
  Allocated reviews: 1.70
  Percentage of total reviews: 0.02%

Product Area: Baby sleep textiles
  Selected Article ID: 00028508
  Actual review count: 137
  Allocated reviews: 3.73
  Percentage of total reviews: 0.04%

Product Area: Bags and travel accessories
  Selected Article ID: 99304696
  Actual review count: 29
  Allocated reviews: 29.88
  Percentage of total reviews: 0.33%

Product Area: Bar stools
  Selected Article ID: 70511234
  Actual review count: 34
  Allocated reviews: 38.07
  Percentage of total reviews: 0.42%

Product Area: Bases and slats
  Selected Article ID: 10488255
  Actual review count: 27
  Allocated reviews: 34.64
  Percentage of total reviews: 0.38%

Product Area: Bathmats
  Selected Article ID: 1059605

In [10]:
# 7. Output the list of selected article_ids.
selected_article_ids = [info['art_id'] for info in selected_articles.values()]
print("\nList of selected article_ids:")
print(selected_article_ids)


List of selected article_ids:
['29535737', '00335772', '00028508', '99304696', '70511234', '10488255', '10596052', '70328626', '80193938', '40253081', '70603498', '09007846', '50450123', '00576945', '10497844', '30290611', '00541574', '10265293', '50316064', '00536407', '00444318', '60430021', '20339868', '00421393', '00163290', '70582085', '50179463', '10576898', '60577819', '00372260', '90356171', '10539066', '70591136', '30545716', '40277360', '90504572', '10496038', '90445592', '20233032', '80518701', '40343614', '79183860', '40464045', '30359196', '99502173', '30369906', '00340759', '90578493', '70588898', '39556102', '30513174', '60307605', '10257472', '00522274', '20563937', '99250269', '80359189', '90331473', '50323597', '90416392', '00530877', '90608230', '00462166', '90542381', '20349947', '00263906', '49429571', '90472590', '50556247', '80542089', '50521390', '90587063', '80488916', '30421042', '00368767', '90462242', '00465773', '80460432', '50441930', '50173239', '4048213

In [ ]:
import os
from openai import AzureOpenAI
from utils.getSecret import get_secret

project = "923326131319"
secret  = "derai-azure"
api_key = get_secret(project, secret)

llm_client = AzureOpenAI(
    api_key=api_key,
    api_version="2023-07-01-preview",
    azure_endpoint="https://derai-vision.openai.azure.com/",
)

model = "gpt-4o" 

In [12]:
def get_topics(reviews):
    messages = [
        {
            "role": "system",
            "content": (
                "You are an helpful customer reviews expert that identifies the main topics discussed in a group of reviews.\n"
                "Use singular words unless a plural form is necessary.\n"                
                "Use only one word. 2 or 3 words can be used only when they are part of a composite word and are better to represent the idea of the topic (e.g.: Ease of use).\n"
                "If you identify a verb as a topic, use the noun version (e.g.: use 'Order' instead of 'Ordering').\n"
                "Generalize the topic word; for example, if you encounter 'Saleswoman' or 'Salesman', abstract it to 'Staff'.\n"
                "Provide the output as a comma-separated list of topics with the first letter capitalized.\n"
            ),
        },
        {
            "role": "user",
            "content": (
                "Read the following reviews and generate a maximum of 8 topics that are mentioned in the reviews.\n"
                "If a product does not have many reviews it is also ok to generate less than 8 topics.\n"
                "For each topic that you generate, indicate which reviews mention implicitly or explicitly that topic.\n"
                "ONLY return topics that are mentioned more than once, don't consider topics mentioned only in a couple of reviews.\n"
                "The topic names should be broad and general, for example Quality, Price, etc.\n"
                "The topics could be either nouns that refers to a certain characteristic of the product or spefic features or parts of the product (screws, cookware, etc.)\n"
                "First return all the topics that you identify as a comma-separated list, then for each of them return a few reviews that mention it.\n"
                f"Reviews: {', '.join(reviews)}\n"
                "Topics:"
            ),
        },
    ]

    response = ' '
    
    # Generate the topic word using the language model
    response = llm_client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=1000,
        temperature=0.4,
        n=1,
        stop=None,
    )

    # Extract and return the topic word
    return response.choices[0].message.content.strip()

In [13]:
import json
topic_results = {}
for art_id in selected_article_ids:
    # Extract all reviews for the given article id.
    # (Assuming each row in df corresponds to one review.)
    article_reviews = df[df['art_id'] == art_id]['review_text'].tolist()
    # Get topics from the reviews.
    answer = get_topics(article_reviews)
    answer = answer.split('\n')[0].split(', ')
    topic_results[art_id] = answer
    print(f"\nArticle {art_id} topics:")
    print(answer)

# 9. Save the results to a JSON file.
with open("topics_sets_GT.json", "w") as f:
    json.dump(topic_results, f, indent=2)

print("\nTopic results saved to 'topic_results.json'.")



Article 29535737 topics:
['Comfort', 'Assembly', 'Quality', 'Design', 'Price', 'Cushion', 'Style', 'Value']

Article 00335772 topics:
['Realism', 'Price', 'Arrangement', 'Quality', 'Appearance', 'Eucalyptus', 'Value', 'Vase']

Article 00028508 topics:
['Softness', 'Price', 'Size', 'Shape', 'Pillowcase', 'Toddler', 'Flatness', 'Quality']

Article 99304696 topics:
['Price', 'Size', 'Quality', 'Durability', 'Staff', 'Usage', 'Availability', 'Rivets']

Article 70511234 topics:
['Comfort', 'Sturdy', 'Assembly', 'Design', 'Colour']

Article 10488255 topics:
['Assembly', 'Beam', 'Slats', 'Quality', 'Service', 'Sturdiness', 'Price', 'Mattress']

Article 10596052 topics:
['Softness', 'Design', 'Price', 'Colors', 'Quality', 'Absorbency', 'Durability', 'Non-slip']

Article 70328626 topics:
['Brightness', 'Installation', 'Wiring', 'Assembly', 'Return', 'Vanity', 'Price', 'Design']

Article 80193938 topics:
['Installation', 'Size', 'Style', 'Quality', 'Plumbing', 'Price', 'Design', 'Compatibility'

In [ ]:
import json
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the JSON file with topic results.
# The JSON structure is expected to be: { "article_id": [ "topic1", "topic2", ... ], ... }
with open("topics_sets_GT.json", "r") as f:
    topic_results = json.load(f)

# Define the embedding model.
model = SentenceTransformer("all-MiniLM-L6-v2")

# Set a similarity threshold.
similarity_threshold = 0.7

# Process each article's topics in the JSON structure.
for art_id, topics in topic_results.items():
    # topics is already a list in this JSON structure.
    if not topics:
        continue  # Skip if no topics are found.
    
    # Encode the topics.
    topic_embeddings = model.encode(topics, convert_to_numpy=True)
    
    # Compute the cosine similarity matrix.
    similarity_matrix = cosine_similarity(topic_embeddings)
    
    # Identify and remove similar topics.
    unique_topics = topics.copy()
    for i in range(len(topics)):
        for j in range(i + 1, len(topics)):
            if similarity_matrix[i, j] > similarity_threshold:
                if topics[j] in unique_topics:
                    unique_topics.remove(topics[j])
                    
    deleted_topics_count = len(topics) - len(unique_topics)
    print(f"Article {art_id}: Number of deleted topics: {deleted_topics_count}")
    print(f"Unique topics: {unique_topics}")
    
    # Update the topics for this article.
    topic_results[art_id] = unique_topics

# Save the updated results to a new JSON file.
with open("topic_results_unique.json", "w") as f:
    json.dump(topic_results, f, indent=2)

print("\nUpdated topic results saved to 'topic_results_unique.json'.")


/Users/davide.zanutto1/Desktop/ratings-reviews-ai-summaries-topics/ratings-reviews/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Article 29535737: Number of deleted topics: 0
Unique topics: ['Comfort', 'Assembly', 'Quality', 'Design', 'Price', 'Cushion', 'Style', 'Value']
Article 00335772: Number of deleted topics: 0
Unique topics: ['Realism', 'Price', 'Arrangement', 'Quality', 'Appearance', 'Eucalyptus', 'Value', 'Vase']
Article 00028508: Number of deleted topics: 0
Unique topics: ['Softness', 'Price', 'Size', 'Shape', 'Pillowcase', 'Toddler', 'Flatness', 'Quality']
Article 99304696: Number of deleted topics: 0
Unique topics: ['Price', 'Size', 'Quality', 'Durability', 'Staff', 'Usage', 'Availability', 'Rivets']
Article 70511234: Number of deleted topics: 0
Unique topics: ['Comfort', 'Sturdy', 'Assembly', 'Design', 'Colour']
Article 10488255: Number of deleted topics: 0
Unique topics: ['Assembly', 'Beam', 'Slats', 'Quality', 'Service', 'Sturdiness', 'Price', 'Mattress']
Article 10596052: Number of deleted topics: 0
Unique topics: ['Softness', 'Design', 'Price', 'Colors', 'Quality', 'Absorbency', 'Durability', 'N

## LLM 3-shot learning - GPT-4o

In [ ]:
llm_client = AzureOpenAI(
    api_key=api_key,
    api_version="2023-07-01-preview",
    azure_endpoint="https://derai-vision.openai.azure.com/",
)

model = "gpt-4o-mini" 

In [16]:
def get_reviews_labels_3shots(review, topics):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful customer reviews expert that identifies the main topics in a review.\n"
                "Provide the output as a comma-separated list of topics. The first letter of the topics should always be capitalized.\n"
            ),
        },
        # Example 1
        {
            "role": "user",
            "content": (
                "Read the following review and associate the topics mentioned implicitly or explicitly in the review.\n"
                "Only answer with the topics that are mentioned in the review. Example: ['price', 'quality']. \n"
                "If you cannot identify any topics, just return '[]' \n"
                "Review: 'The product arrived quickly, and the packaging was great. However, the price is too high.'\n"
                "Topics: ['Delivery', 'Packaging', 'Price', 'Quality']. DO NOT write any topics outside of this list. \n"
                "Topics mentioned within the review:"
            ),
        },
        {
            "role": "assistant",
            "content": "['Delivery', 'Packaging', 'Price']",
        },
        # Example 2
        {
            "role": "user",
            "content": (
                "Read the following review and associate the topics mentioned implicitly or explicitly in the review.\n"
                "Only answer with the topics that are mentioned in the review. Example: ['price', 'quality']. \n"
                "If you cannot identify any topics, just return '[]' \n"
                "Review: 'I love how comfortable these shoes are! They fit perfectly, and the material feels premium. Good price.'\n"
                "Topics: ['Comfort', 'Fit', 'Material', 'Design', 'Value']. DO NOT write any topics outside of this list.\n"
                "Topics mentioned within the review:"
            ),
        },
        {
            "role": "assistant",
            "content": "['Comfort', 'Fit', 'Material', 'Value']",
        },
        # Example 3
        {
            "role": "user",
            "content": (
                "Read the following review and associate the topics mentioned implicitly or explicitly in the review.\n"
                "Only answer with the topics that are mentioned in the review. Example: ['price', 'quality']. \n"
                "If you cannot identify any topics, just return '[]' \n"
                "Review: 'The app crashes frequently and is very slow. It needs major improvements.'\n"
                "Topics: ['Performance', 'Usability', 'Design']. DO NOT write any topics outside of this list. \n"
                "Topics mentioned within the review:"
            ),
        },
        {
            "role": "assistant",
            "content": "['Performance', 'Usability']",
        },
        {
            "role": "user",
            "content": (
                "Read the following review and associate the topics mentioned implicitly or explicitly in the review.\n"
                "Only answer with the topics that are mentioned in the review. Example: ['Price', 'Quality']. \n"
                "If you cannot identify any topics, just return '[]' \n"
                f"Review: '{review}' \n"
                f"Topics: {topics}. DO NOT write any topics outside of this list. \n"
                f"Topics mentioned within the review:"
            ),
        },
    ]

    response = ' '
    model = "gpt-4o-mini" 
    # Generate the topic word using the language model
    response = llm_client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=30,
        temperature=0.4,
        n=1,
        stop=None,
    )

    # Extract and return the topic word
    return response.choices[0].message.content.strip()

In [17]:
print(topics)

['Installation', 'Screws', 'Design', 'Fit', 'Price', 'Space', 'Weight', 'Material']


In [ ]:
import logging
import os
import json
import pandas as pd
import ast

# Define the output CSV filename.
filename = 'csv/GroundTruthProdArea10kV2.csv'

# Load the JSON file with article topics.
# The JSON structure is assumed to be like:
# {
#   "30552226": ["Comfort", "Assembly", "Cushion", ...],
#   "80335773": ["Quality", "Realism", "Price", ...]
# }
topics_filename = "topic_results_unique.json"
with open(topics_filename, "r") as f:
    article_topics = json.load(f)


# Assume df is your DataFrame with fields "art_id" and "review_text".
# For example, you might have loaded it like:
# df = pd.read_csv("your_reviews_file.csv")

# Set the minimum number of reviews required for a topic to be retained.
min_topic_reviews = 2

# This list will collect rows in the format: [article_id, review, all_topics, selected_topics]
results = []

# Group the DataFrame by "art_id".
grouped = df.groupby("art_id")
for article_id, group in grouped:
    # Retrieve topics for this article (using str() if JSON keys are strings).
    topics = article_topics.get(str(article_id), [])
    if not topics:
        logging.error(f"No topics found for article {article_id}. Skipping this article.")
        continue
    
    # List to store results for the current article.
    article_results = []
    
    # Process each review in the group.
    for idx, row in group.iterrows():
        review = row["review_text"]
        try:
            # Call your function to get selected topics from the review.
            # get_reviews_labels_3shots should be defined elsewhere.
            selected_topics_str = get_reviews_labels_3shots(review, topics)
            # If selected_topics_str is already a list, this conversion should be skipped.
            # You can add a check like:
            if isinstance(selected_topics_str, list):
                selected_topics = selected_topics_str
            else:
                selected_topics = ast.literal_eval(selected_topics_str)
            article_results.append([review, topics, selected_topics])
        except Exception as e:
            # If an exception related to content filtering occurs, log it and save empty topics.
            if "content_filter" in str(e) or "ResponsibleAIPolicyViolation" in str(e):
                logging.error(f"Content filter triggered for review: {review} - Skipping.")
            else:
                logging.error(f"Error processing review: {review} - {e}")
            article_results.append([review, [], []])
    
    # Count the frequency of each topic in the selected topics for this article.
    topic_frequency = {}
    for item in article_results:
        sel_topics = item[2]
        for t in sel_topics:
            topic_frequency[t] = topic_frequency.get(t, 0) + 1
        
    
    # Filter topics: only keep topics that appear in at least min_topic_reviews reviews.
    filtered_topics = [t for t in topics if topic_frequency.get(t, 0) >= min_topic_reviews]
    # Update each result row to remove topics that are not in filtered_topics.
    for item in article_results:
        # item[1] holds the original topics passed to the function.
        # item[2] holds the selected topics returned by the function.
        item[1] = [t for t in item[1] if t in filtered_topics]
        item[2] = [t for t in item[2] if t in filtered_topics]
        results.append([article_id] + item)

# Create a DataFrame from the results with columns: article_id, review, all_topics, selected_topics.
result_df = pd.DataFrame(results, columns=["article_id", "review", "all_topics", "selected_topics"])

# If the CSV file already exists and is non-empty, append the new data.
if os.path.exists(filename) and os.path.getsize(filename) > 0:
    existing_df = pd.read_csv(filename)
    result_df = pd.concat([existing_df, result_df], ignore_index=True)

# Save the final DataFrame to CSV.
result_df.to_csv(filename, index=False)
print(f"Results saved to {filename}")

ERROR:root:No topics found for article 00030262. Skipping this article.
ERROR:root:No topics found for article 00031841. Skipping this article.
ERROR:root:No topics found for article 00038159. Skipping this article.
ERROR:root:No topics found for article 00057256. Skipping this article.
ERROR:root:No topics found for article 00058487. Skipping this article.
ERROR:root:No topics found for article 00069768. Skipping this article.
ERROR:root:No topics found for article 00081534. Skipping this article.
ERROR:root:No topics found for article 00086428. Skipping this article.
ERROR:root:No topics found for article 00089163. Skipping this article.
ERROR:root:No topics found for article 00091415. Skipping this article.
ERROR:root:No topics found for article 00098959. Skipping this article.
ERROR:root:No topics found for article 00100467. Skipping this article.
ERROR:root:No topics found for article 00101056. Skipping this article.
ERROR:root:No topics found for article 00102065. Skipping this a